## Script to bundle all resources in folder as a batch, transaction, collection, or message

- python version 3.6+
- get data from folder
- bundle
- validate and save

In [93]:
from json import load, dumps, loads
from yaml import safe_load as yloads, safe_dump as ydumps
from IPython import display as D
from requests import get, post, put
from IPython.display import display, Markdown, HTML
from fhir.resources.bundle import Bundle as B, BundleEntry as BE, BundleEntryRequest as BER, BundleEntryResponse as BERes, BundleEntrySearch as BES 
from fhir.resources.meta import Meta as M
from fhir.resources.extension import Extension as X
import uuid
from datetime import datetime
from pprint import pprint
from pathlib import Path

#===============bundle variables=================
bundle_id = 'cdex-scenario7-payer-load-transaction-bundle'
bundle_type = 'transaction' #'collection'
transaction_method = 'POST' # 'PUT' or 'POST'
title = False #True # if True add meta extension title and description to bundle
bundle_description = 'This is an Bundle example for the **Patient Access Brands Bundle Profile**. and represents Example 5 - EHR and EHR Customer Hosted Brands Bundles'
bundle_profile = 'http://hl7.org/fhir/smart-app-launch/StructureDefinition/patient-access-brands-bundle'
last_updated = True # if True add last updated to bundle


keep_ids = False #True # if False strip FHIR ids from all resources
use_uuid = True #False # if False use references
base = 'https://ehr.example.org'  #  if using reference instead of UUIDs

in_folder = '/Users/ehaas/Documents/Python/Jupyter/MyNotebooks/bundler/r4/CDex_PAS_Payer_Load'  # copy/paste absolute path
in_path = Path(in_folder)

in_path

PosixPath('/Users/ehaas/Documents/Python/Jupyter/MyNotebooks/bundler/r4/CDex_PAS_Payer_Load')

###  write to file

In [94]:
def write_file(r_dict): # write file
    out_path = in_path / f'Bundle-{bundle_id}.yml'
    print(out_path)
    out_path.write_text(ydumps(r_dict, sort_keys=False))  # dump bundle to file

### open json file and return as dict

In [95]:


b = B(
    id = bundle_id,
    type = bundle_type,
    timestamp = datetime.now().astimezone().isoformat(),
)

meta = M()
print(meta)
meta.profile = [bundle_profile] if bundle_profile else None

if title:
    
    meta.extension = []
    meta.extension.append ({ 'url': 'http://hl7.org/fhir/StructureDefinition/instance-name', "valueString": f'Bundle {bundle_id.title()}'})
    meta.extension.append({ 'url': 'http://hl7.org/fhir/StructureDefinition/instance-description', "valueMarkdown": bundle_description})

if last_updated:
    meta.lastUpdated = datetime.now().astimezone().isoformat()
    
b.meta = meta
b.json()


resource_type='Meta' fhir_comments=None extension=None id=None lastUpdated=None lastUpdated__ext=None profile=None profile__ext=None security=None source=None source__ext=None tag=None versionId=None versionId__ext=None


'{"resourceType":"Bundle","id":"cdex-scenario7-payer-load-transaction-bundle","meta":{"lastUpdated":"2023-09-10T07:46:52.632979-07:00","profile":["http://hl7.org/fhir/smart-app-launch/StructureDefinition/patient-access-brands-bundle"]},"type":"transaction","timestamp":"2023-09-10T07:46:52.632632-07:00"}'

In [96]:
def open_file(f_name): # get files
    
    try:
        return(loads(f_name.read_text()))
    except ValueError as e:

        return(yloads(f_name.read_text()))

### Get files in current path

In [97]:
files = [path for path in in_path.iterdir() if 'Bundle' not in path.stem]
for file in files:
    print(file.name)

PractitionerRole-referral-example.yml
example-1-patient.json
ServiceRequest-referralrequest-example.yml
Practitioner-referral-example.yml
Coverage-insurance-example.yml
Location-referral-example.yml
Organization-UMO-example.yml
cdex-claim-MRIAuthorizationRIExample.yml
Organization-insurer-example.yml


### create Bundle 'b'  change the id for unique Bundles!!!

### Add resources to bundle

#### Create a mapping of ids to uuids

- get id from resource
- map to uuid (create a dict of {urn, (old_ref)})
- remove id element
- replace all old refs in bundle with new urns


In [98]:
b.entry = []
ref_map = {}
for i in sorted(files):
    r = open_file(i)
    # remove text,id, meta.extension elements from file 
    try:
        r["meta"].pop('extension', None)
    except KeyError:
        pass
    r.pop('text', None) # remove id elements from files
    if keep_ids:
        old_id = r['id'] # map old id to new urn
    else:
        old_id = r.pop('id',None) # map old id to new urn
    if use_uuid:
        try:
            print(r['url'])
            new_urn = r['url']
        except:
            new_urn = uuid.uuid1().urn # new urn for resource
        ref_map[new_urn] = f'{r["resourceType"]}/{old_id}'
        # print( id_map , '\n')

        r['id'] = new_urn.split(':')[-1] if keep_ids else None # update id to new urn}
        e = dict( 
            fullUrl = new_urn,
            resource = r,
        )
    else: # use resources assume the references the ids are correct
        e = dict( 
            fullUrl = f'{base}/{r["resourceType"]}/{old_id}',
            resource = r,
        )
    print(e)
    if bundle_type in ['transaction', 'batch']:
        e['request'] = dict(
                    method = transaction_method,
                    url = r['resourceType']
                    )
                    
    # if for a PAS request or response Bundle
    if r['resourceType'] in ['ClaimResponse','Claim']:
        b.entry.insert(0,e)
    else:
        b.entry.append(e)
b_json = b.json()
if use_uuid:

    for new_ref, old_ref in ref_map.items():
        b_json = b_json.replace(old_ref,new_ref)
    # print(b_json)
    b_obj = B.parse_raw(b_json)

print(b_json)

write_file(loads(b_json))


{'fullUrl': 'urn:uuid:e154b866-4fe8-11ee-9291-daaf18ab2cbb', 'resource': {'resourceType': 'Coverage', 'meta': {'profile': ['http://hl7.org/fhir/us/davinci-pas/StructureDefinition/profile-coverage']}, 'status': 'active', 'beneficiary': {'reference': 'Patient/SubscriberExample'}, 'payor': [{'reference': 'Organization/InsurerExample'}], 'id': None}}
{'fullUrl': 'urn:uuid:e154ea02-4fe8-11ee-9291-daaf18ab2cbb', 'resource': {'resourceType': 'Location', 'meta': {'profile': ['http://hl7.org/fhir/us/davinci-pas/StructureDefinition/profile-locaation']}, 'name': 'REFERRAL CLINIC', 'address': {'line': ['111 1ST STREET'], 'city': 'SAN DIEGO', 'state': 'CA', 'postalCode': '92101', 'country': 'US'}, 'id': None}}
{'fullUrl': 'urn:uuid:e155488a-4fe8-11ee-9291-daaf18ab2cbb', 'resource': {'resourceType': 'Organization', 'meta': {'profile': ['http://hl7.org/fhir/us/davinci-pas/StructureDefinition/profile-requestor']}, 'identifier': [{'system': 'http://hl7.org/fhir/sid/us-npi', 'value': '8189991234'}], 'ac